In [ ]:
%flink.ssql

CREATE TABLE input_stream_kinesis (
  nome STRING,
  valor DOUBLE,
  `timestamp` TIMESTAMP(3),
  texto STRING,
  WATERMARK FOR `timestamp` AS `timestamp` - INTERVAL '5' SECOND
) WITH (
  'connector' = 'kinesis',
  'stream' = 'kinesis-data-streams-projeto-27abd',
  'aws.region' = 'us-east-1',
  'scan.stream.initpos' = 'LATEST',
  'format' = 'json',
  'json.timestamp-format.standard' = 'ISO-8601'
);

In [ ]:
%flink.ssql
SELECT 
  window_start,
  window_end,
  nome,
  round(soma_valor) as soma_valor_ultima_janela
FROM (
  SELECT
    TUMBLE_START(PROCTIME(), INTERVAL '10' SECOND) AS window_start,
    TUMBLE_END(PROCTIME(), INTERVAL '10' SECOND) AS window_end,
    nome,
    SUM(valor) AS soma_valor
  FROM input_stream_kinesis
--   WHERE nome = 'Gabriel'
  GROUP BY
    TUMBLE(PROCTIME(), INTERVAL '10' SECOND),
    nome
)
ORDER BY window_end DESC
LIMIT 5;